# 👨‍💻🔍 EDA Topics des Articles

Ce notebook permet d'explorer les possibilités liées à la tâche Notion : Classifier le topic des articles afin de pouvoir comparer les sujets.

In [2]:
import sys
sys.path.append('..')
from quotaclimat.data_processing.sitemap.sitemap_processing import load_all

import pandas as pd

In [4]:
df = load_all("../data_public/sitemap_dumps/")
print(df.shape)
print(df.columns)
df.head()

(320545, 26)
Index(['url', 'news', 'news_publication', 'publication_name',
       'publication_language', 'news_publication_date', 'news_title',
       'news_keywords', 'image', 'image_loc', 'image_caption', 'sitemap',
       'etag', 'sitemap_last_modified', 'sitemap_size_mb', 'download_date',
       'media', 'section', 'changefreq', 'news_access', 'image_title',
       'lastmod', 'news_genres', 'priority', 'download_date_last',
       'media_type'],
      dtype='object')


,url,news,news_publication,publication_name,publication_language,news_publication_date,news_title,news_keywords,image,image_loc,image_caption,sitemap,etag,sitemap_last_modified,sitemap_size_mb,download_date,media,section,changefreq,news_access,image_title,lastmod,news_genres,priority,download_date_last,media_type
37884,https://www.francetvinfo.fr/monde/nouvel-an-20...,\n,\n,Franceinfo,fr,2022-12-31 15:57:46,Nouvel An 2023 : le feu d’artifice fait son re...,"Nouvel An 2023, Monde, France",\n,https://www.francetvinfo.fr/image/761i6v10w-6c...,Nouvel An 2023 : le feu d’artifice fait son re...,https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[monde, nouvel-an-2021]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37885,https://www.francetvinfo.fr/replay-jt/france-3...,\n,\n,Franceinfo,fr,2022-12-31 16:03:28,"Nouvel An 2023 : dans la Marne, des éthylotest...","Nouvel An 2023, Monde, Sécurité routière, Mode...",\n,https://www.francetvinfo.fr/image/761i6v4km-1a...,"Nouvel An 2023 : dans la Marne, des éthylotest...",https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[replay-jt, france-3, 12-13]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37886,https://www.francetvinfo.fr/culture/charente-m...,\n,\n,Franceinfo,fr,2022-12-31 16:05:38,Charente-Maritime : découverte de la ferme des...,"Culture, Animaux",\n,https://www.francetvinfo.fr/image/761i6v4sd-70...,Charente-Maritime : découverte de la ferme des...,https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,[culture],None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37887,https://www.francetvinfo.fr/monde/nouvel-an-20...,\n,\n,Franceinfo,fr,2022-12-31 16:00:23,Nouvel An 2023 : un dispositif de sécurité imp...,"Nouvel An 2023, Monde",\n,https://www.francetvinfo.fr/image/761i6v19q-42...,Nouvel An 2023 : un dispositif de sécurité imp...,https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[monde, nouvel-an-2021]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
37888,https://www.francetvinfo.fr/replay-radio/micro...,\n,\n,Franceinfo,fr,2022-12-31 15:42:02,La fondation René Cassin : développer les droi...,"Europe, Mode de vie, lifestyle, Formation, éco...",\n,https://www.francetvinfo.fr/image/761i6wgbd-5e...,"Le juriste, diplomate et homme politique franç...",https://www.francetvinfo.fr/sitemap_news.xml,"""059a07258cabf77ded25d09acd8d9ff3d7e01e84""",2023-01-01 01:46:04+00:00,0.216074,2023-01-01 01:48:02,francetvinfo,"[replay-radio, micro-europeen]",None,None,None,None,None,NaN,2023-01-02 01:28:01,tv
